In [19]:
import torch
import torch.nn.functional as F
import numpy as np
import json
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from torch_geometric.nn import GCNConv
import random

random_seed = 42

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)



# Load data
adj = sp.load_npz("adj.npz") # Defines graph architecture
feat = np.load("features.npy") # Defines nodes feature
labels = np.load("labels.npy") 

splits = json.load(open('splits.json'))
idx_train0, idx_test = splits['idx_train'], splits['idx_test']

# Split train data into train and validation sets
idx_train, idx_val = train_test_split(idx_train0, test_size=0.15, random_state=42)

# Convert to PyG format



edge_index, _ = from_scipy_sparse_matrix(adj)
x = torch.tensor(feat, dtype=torch.float)
y = torch.full((x.size(0),), -1, dtype=torch.long)
y[idx_train0] = torch.tensor(labels, dtype=torch.long)
# y = torch.tensor(labels, dtype=torch.long)
train_mask = torch.zeros(x.size(0), dtype=torch.bool)
train_mask[idx_train] = True
val_mask = torch.zeros(x.size(0), dtype=torch.bool)
val_mask[idx_val] = True
test_mask = torch.zeros(x.size(0), dtype=torch.bool)
test_mask[idx_test] = True

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask)
data

Data(x=[2480, 1390], edge_index=[2, 10100], y=[2480], train_mask=[2480], val_mask=[2480], test_mask=[2480])

In [20]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden1, num_hidden2, num_classes):
        super().__init__()
        self.conv1 = GCNConv(num_node_features, num_hidden1)
        self.conv2 = GCNConv(num_hidden1, num_hidden2)
        self.conv3 = GCNConv(num_hidden2, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)


from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

In [22]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

def train_and_evaluate(data, model, optimizer, patience=10):
    best_val_loss = float('inf')
    counter = 0

    for epoch in range(200):
        model.train()
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = F.nll_loss(out[data.val_mask], data.y[data.val_mask])

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1

        if counter >= patience:
            break

    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
    acc = int(correct) / int(data.val_mask.sum())

    return acc

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
import numpy as np

num_random_search = 20

param_grid = {
    'lr': [0.01, 0.03, 0.05],
    'weight_decay': [1e-4 ,1e-3, 1e-2],
    'num_hidden1': [16 ,32, 64, 128],
    'num_hidden2': [16 ,32, 64, 128]

}

for i in range(num_random_search):
    lr = np.random.choice(param_grid['lr'])
    weight_decay = np.random.choice(param_grid['weight_decay'])
    num_hidden1 = np.random.choice(param_grid['num_hidden1'])
    num_hidden2 = np.random.choice(param_grid['num_hidden2'])

    avg_acc = 0
    for train_idx, val_idx in kf.split(idx_train0):
        idx_train, idx_val = train_idx, val_idx
        train_mask = torch.zeros(x.size(0), dtype=torch.bool)
        train_mask[idx_train] = True
        val_mask = torch.zeros(x.size(0), dtype=torch.bool)
        val_mask[idx_val] = True

        data.train_mask = train_mask
        data.val_mask = val_mask

        model = GCN(num_node_features=data.x.shape[1], num_hidden1=num_hidden1, num_hidden2=num_hidden2, num_classes=data.y.max().item() + 1).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        acc = train_and_evaluate(data, model, optimizer)
        avg_acc += acc

    avg_acc /= n_splits
    print(f"Random Search Iteration {i + 1}, lr: {lr}, weight_decay: {weight_decay}, num_hidden: {num_hidden}, Avg. Accuracy: {avg_acc:.4f}")

    if avg_acc > best_acc:
        best_acc = avg_acc
        best_params = {'lr': lr, 'weight_decay': weight_decay, 'num_hidden': num_hidden}

print(f"Best parameters: {best_params}, Best average accuracy: {best_acc:.4f}")



IndexError: Target -1 is out of bounds.

In [9]:
from skopt import Optimizer
from skopt.utils import use_named_args
from skopt.space import Real, Integer
np.int = int

space = [
    Real(1e-4, 1e-1, "log-uniform", name="lr"),
    Real(1e-4, 1e-2, "log-uniform", name="weight_decay"),
    Integer(8, 128, name="num_hidden")
]

@use_named_args(space)
def objective(**params):
    lr = params["lr"]
    weight_decay = params["weight_decay"]
    num_hidden = params["num_hidden"]

    avg_acc = 0
    for train_idx, val_idx in kf.split(idx_train0):
        idx_train, idx_val = idx_train0[train_idx], idx_train0[val_idx]
        train_mask = torch.zeros(x.size(0), dtype=torch.bool)
        train_mask[idx_train] = True
        val_mask = torch.zeros(x.size(0), dtype=torch.bool)
        val_mask[idx_val] = True

        data.train_mask = train_mask
        data.val_mask = val_mask

        model = GCN(num_node_features=data.x.shape[1], num_hidden=num_hidden, num_classes=data.y.max().item() + 1).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        acc = train_and_evaluate(data, model, optimizer)
        avg_acc += acc

    avg_acc /= n_splits
    print(f"lr: {lr}, weight_decay: {weight_decay}, num_hidden: {num_hidden}, Avg. Accuracy: {avg_acc:.4f}")

    return -avg_acc

num_bayesian_iter = 20
optimizer = Optimizer(space, base_estimator="GP", n_initial_points=5)
best_acc = -np.inf
best_params = None

for i in range(num_bayesian_iter):
    next_x = optimizer.ask()
    f_val = objective(next_x)
    optimizer.tell(next_x, f_val)

    if -f_val > best_acc:
        best_acc = -f_val
        best_params = {'lr': next_x[0], 'weight_decay': next_x[1], 'num_hidden': next_x[2]}

print(f"Best parameters: {best_params}, Best average accuracy: {best_acc:.4f}")



TypeError: only integer scalar arrays can be converted to a scalar index